In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import mpld3
from scipy.fftpack import fft, fftfreq
mpl.style.use('solarized')

# Parametric Audio Loudspeaker

## Why parametric audio?



Panel Array Loudspeaker | Parametric Loudspeaker
-----------------------  | ---------------------
<img src="Presentation/fieldPanel.png" style="width: 250px;"/> | <img src="Presentation/fieldParametric.png" style="width: 250px;"/>


### Simplified Equation

Khokhlov, Zabolotskaya, and Kuznetsov ~ 1970

$$
\frac{\partial^2 p}{\partial z \partial \tau} 
= 
\frac{c_0}{2} \nabla_r^2 p
+
\frac{\delta}{2c_0^3} \frac{\partial^3 p}{\partial\tau^3}
+
\frac{\beta}{2\rho_0 c_0^3} \frac{\partial^2 p^2}{\partial\tau^2}
$$

### Amplitude Modulation

$\begin{align}
p(t) = E(t) \sin(\omega_c t)
\end{align}$

$\begin{align}
p_{aud} \propto \frac{d^2}{dt^2} E^2(t)
\end{align}$

$\begin{align} 
E(t) = \sqrt{1+\iint s(t)dt^2} 
\end{align}$

### Sidebands

In [ ]:
f0=1e3
fc=40e3
fs=192e3
periods = 100
nsamples = np.ceil(periods*fs/f0).astype('int')
#nsamples = (2**np.ceil(np.log2(nsamples))).astype('int')
t=np.arange(nsamples)/fs

carrier = np.sin(2*np.pi*t*fc)
audio = np.sin(2*np.pi*t*f0)
envelope = np.sqrt(1+audio)
#envelope = 1
ultrasound = envelope*carrier

ultraspectrum = fft(ultrasound)
f = fftfreq(nsamples,1/fs)

fidx = (f>0) & (f<fs/2)

plt.plot(f[fidx],20*np.log10(np.abs(ultraspectrum[fidx])))
plt.ylim(-20,80)
plt.xlabel('Frequency in Hz')
plt.ylabel('Pressure in dB')
mpld3.display()
#plt.show()


## Nonlinear Identification

### Volterra Series
$\begin{align}
y(t) = \sum_{n=1}^\infty \int \ldots \int 
h_n(\tau_1,\ldots,\tau_n) 
\prod_{j=1}^n x(t-\tau_j) d\tau_j
\end{align}$

### Hammerstein System
<img src="Presentation/hammerstein.pdf" style="width: 600px;"/>
$\begin{align}
y(t) = \int h(t-\tau)\, m(x(\tau)) \, d\tau
\end{align}$

### Weiner System
<img src="Presentation/weiner.pdf" style="width: 600px;"/>

$\begin{align}
y(t) = m\left( \int h(t-\tau)\, x(\tau) \, d\tau \right) 
\end{align}$


### Approximation

In [ ]:
from nonlinear import hammersteinApproximation as hammerstein
from nonlinear import weinerApproximation as weiner
from scipy.interpolate import interp1d
nsamples = 2**12
noiselevel = 0.25

def m(u):
    #return (1-np.exp(-np.abs(u)))*np.sign(u)
    return u**2
m = np.vectorize(m)
    
def lbda(n):
    if n < 0:
        return 0.0
    elif n > 1000:
        return 0.0
    else:
        return 1.0/1.2**n

lbda = np.vectorize(lbda)

inputsignal = 2*np.random.rand(nsamples)-1
hammerout = np.zeros(nsamples)

for n in range(nsamples):
    hammerout[n] = np.sum( m(inputsignal[0:n+1]) * lbda(np.r_[n:-1:-1]) )
hammernoise = hammerout + noiselevel*np.max(np.abs(hammerout))*np.random.randn(nsamples)
scale = np.max(np.abs(hammernoise))
hammernoise = hammernoise/scale
hammerPoints, mu = hammerstein(inputsignal,hammernoise,
                               h=lambda n:n**(-0.3),npoints=int(nsamples/4))
finIdx = np.isfinite(mu)
muFun = interp1d(hammerPoints[finIdx],mu[finIdx])

In [ ]:
plt.scatter(inputsignal, hammernoise, 
            marker='.', facecolor='none', edgecolor='gray', alpha=0.5)
plt.plot(hammerPoints, m(hammerPoints), label='True nonlinearity')
plt.plot(hammerPoints, muFun(hammerPoints)*scale, label='Approximate nonlinearity')
plt.legend()
plt.xlabel('Input samples')
plt.ylabel('Output samples')
plt.xlim(-1,1)
#plt.show()
mpld3.display()

In [ ]:
%qtconsole
